In [101]:
import re
import subprocess

from astropy.io import fits
import pyfits

In [100]:
ls


29-06-2014UT-ARCSAT/  dataSet.py  LICENSE         masterlog.log
Basic-Notebook.ipynb  images.py   log_handler.py  README.md


In [880]:
def read_obs_log(logName):
    # The Latin-1 encoding is needed because of the degree symobl.
    log = open(logName,'r', encoding='Latin-1')
    log = log.read()
    return log

def extract_log_metadata(log):
    """
    Parse log string and return a list of filenames and a dictionary.
    
    Arguments:
    - ARCSAT log in the form of a sngle string.
    
    Returns:
    - List of filenames(strings) that apear in the log. The file names
      have no .fits extension.
    - Dictionary whith the filenames as keys. The value for each filename
      (key) is a dictionary with the time. 
      
    Parses an arcsat log in the form of a single string; meant to handle
    a string comprised of all the logs for a given night. Finds all image
    file names referanced in the log and creates a list of those names (these
    file names don't have .fits extensions). Builds a dictionary with keys for
    each of these files. The value for each is itself a dictions with keys and
    vlues coresponding to information in the log. Find the time an image was
    finished and weither or not the image was plate solved. If plate solved
    the average FWHM is found.
    """
    
    filenames = []
    filesMeta = {}
    # Find the files that have information in the logs.
    searches = re.findall(r'(Imaging to) (\S+)', log)
    [filenames.append(x[1].strip('\n')) for x in searches[:]]
    # Find all the relevant information on each file in the logs.
    for name in filenames:
        # Build a string containing only information for a single file.
        searchstring = r''+name+'.*?turning tracking off for safety\)'
        singleImageMeta = str(re.findall(searchstring, log, re.DOTALL)[0])
        filesMeta[name] = {}
        # Find the time that a given file was finished.
        finishTime = re.search(r'(\(exposure complete and image downloaded\))(.*?)(\S{8})',
                               singleImageMeta, re.DOTALL)
        finishTime = finishTime.group(3)
        filesMeta[name]['finishTime'] = finishTime
        # Dtermine if an image was plate solved and if so, get the average FWHM.
        if 'Solved!' in singleImageMeta:
            filesMeta[name]['solved'] = True
            fwhm = re.search(r'(Image) (FWHM is) (\d\D\d)', singleImageMeta)
            fwhm = float(fwhm.group(3))
            filesMeta[name]['fwhm'] = fwhm
        else:
            filesMeta[name]['solved'] = False
    return filenames, filesMeta


def compare_log_to_files(lognames):
    samenames = []
    diffnames = []
    darks = []
    biases = []
    
    ls = subprocess.check_output(['ls -1 *.fits'],
                                 universal_newlines=True, shell=True).strip().split('\n')
    for i in ls:
        
        i = i.replace('.fits','')
        if i in lognames:
            samenames.append(i)
        else:
            diffnames.append(i)
    return ls, samenames, diffnames

In [875]:
cd /home/brandon/Work/ADRL/

/home/brandon/Work/ADRL


In [876]:
log = read_arcsat_obs_log('masterlog.log')

In [877]:
cd 29-06-2014UT-ARCSAT/

/home/brandon/Work/ADRL/29-06-2014UT-ARCSAT


In [882]:
fileNames, filesMeta = extract_log_metadata(log)
ls, same, diff = compare_log_to_files(filenames)
print(diff)
print(ls)

['Bias_B2_20140629_115905', 'Bias_B2_20140629_115917', 'Bias_B2_20140629_115927', 'Bias_B2_20140629_115938', 'Bias_B2_20140629_115948', 'Bias_B2_20140629_115958', 'Bias_B2_20140629_120009', 'Bias_B2_20140629_120019', 'Bias_B2_20140629_120030', 'Bias_B2_20140629_120040', 'Bias_B2_20140629_120051', 'Dark_B2_20140629_120359', 'Dark_B2_20140629_120729', 'Dark_B2_20140629_121100', 'domeflat_halpha_6563_50_001', 'domeflat_johnson-cousins_Ic_001', 'domeflat_johnson-cousins_Ic_002', 'domeflat_johnson-cousins_Ic_003', 'domeflat_johnson-cousins_Ic_004', 'domeflat_johnson-cousins_Ic_005', 'domeflat_johnson-cousins_Ic_006', 'domeflat_johnson-cousins_Ic_007', 'domeflat_johnson-cousins_Ic_008', 'domeflat_johnson-cousins_Ic_009', 'domeflat_johnson-cousins_Ic_010', 'domeflat_johnson-cousins_Ic_011', 'domeflat_johnson-cousins_Rc_001', 'domeflat_johnson-cousins_Rc_002', 'domeflat_johnson-cousins_Rc_003', 'domeflat_johnson-cousins_Rc_004', 'domeflat_johnson-cousins_Rc_005', 'domeflat_johnson-cousins_Rc_0